### Sourcing SCOTUS from Harvard's [Caselaw Access Project (CAP)](https://case.law/)

Goal: retrieve all opinions written by the Supreme Court for a specified year range.

SCOTUS denies thousands of cases every year, and each denial gets its own document, so we can't just grab all SCOTUS documents from CAP for a specified year. We need docket numbers for the cases that granted cert and argued before the court. Here, we source those docket numbers from the [Super-SCOTUS dataset](https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/POWQIT) [[paper](https://aclanthology.org/2023.nllp-1.20/)].

1. Get docket numbers for the years 1986-2019 from superscotus.
2. For each year, request a small sample (~15) cases from CAP. (waiting on unmetered API access before pulling full set)


There also seem to be two maps from SCDB to CAP:
1. [Connecting U.S. Supreme Court Case Information and Opinion Authorship (SCDB) to Full Case Text Data (CAP), 1791-2011](https://zenodo.org/records/4344917).
1. CAP's own [matchup file](https://case.law/download/scdb/scdb_matchup_2020-01-16.csv) on this [page](https://case.law/download/scdb/).

The maps from SCDB end in 2011. From 2007 onward, the SCOTUS website has documents with docket numbers for cases granted argument. These are the **Granted \& Noted** [lists](https://www.supremecourt.gov/orders/grantednotedlists.aspx).

In [ ]:
# TODO: Scriptify this notebook

In [1]:
%cd -q ../..

In [ ]:

import os
import sqlite3
from collections import defaultdict
from itertools import chain

import aiohttp
import jsonlines
from dotenv import load_dotenv

from scotus_metalang.diachronic_analysis import cap

load_dotenv()
CAP_TOKEN = os.environ["CAP_TOKEN"]

docket_nums_by_year = defaultdict(list)
with jsonlines.open("data/super_scotus/1986_to_2019.jsonl", "r") as f:
    for case in f:
        # Example case id: "1986_84-2022"
        year = case["year"]
        docket_number = case["id"][5:]
        docket_nums_by_year[year].append(docket_number)

docket_numbers = list(chain(*docket_nums_by_year.values()))
docket_numbers = [{"docket_number": x} for x in docket_numbers]


In [2]:
connection = sqlite3.connect("api_log.db")
# Insert all docket numbers into cases
with connection:
    connection.executemany("""--sql
                           INSERT OR IGNORE INTO cases (docket_number)
                           VALUES(:docket_number)
                           """, docket_numbers)
# Figure out which docket numbers need to be processed
with connection:
    rows = connection.execute("""--sql
                              SELECT * FROM cases
                              WHERE case_status != 'success' OR case_status IS NULL
                              """).fetchall()
docket_numbers_to_process = [row[0] for row in rows]

len(docket_numbers_to_process)

2853

In [5]:
async def main():
    connector = aiohttp.TCPConnector(limit_per_host=10)
    headers={"Authorization": f"Token {CAP_TOKEN}"}
    async with aiohttp.ClientSession(connector=connector, headers=headers) as session:
        for i, docket_number in enumerate(docket_numbers_to_process):  # Sample here to limit API usage while tinkering
            db_params, opinions_as_params = await cap.process_opinions_by_docket_number(docket_number, session)
            with connection:
                connection.execute("""--sql
                                   UPDATE cases
                                   SET case_status = :case_status,
                                   selected_case_id = :selected_case_id,
                                   decision_date = :decision_date
                                   WHERE docket_number = :docket_number
                                   """, db_params)
                connection.executemany("""--sql
                                       INSERT INTO opinions
                                       VALUES(:docket_number, :opinion_number, :cap_author, :author)
                                       """, opinions_as_params)

In [ ]:
# Docket numbers to exclude on potential rerun
with connection:
    rows = connection.execute("""--sql
                             SELECT docket_number
                             FROM opinions
                             GROUP BY docket_number
                             HAVING cap_author is not null
                             """).fetchall()

docket_numbers_to_exclude = [row[0] for row in rows]
docket_numbers_to_exclude

docket_nums = list(set(docket_numbers_to_process) - set(docket_numbers_to_exclude))